In [5]:
from google.colab import drive
drive.mount('/content/Google_Drive')

Drive already mounted at /content/Google_Drive; to attempt to forcibly remount, call drive.mount("/content/Google_Drive", force_remount=True).


In [6]:
import cv2 as cv
import glob as gb
import matplotlib.pyplot as plt
import csv 
from collections import OrderedDict

path=r'/content/Google_Drive/MyDrive/College /Human Pose Estimation Using Open CV/Graph/graph.pb'
net= cv.dnn.readNetFromTensorflow(path)
path =gb.glob('/content/Google_Drive/MyDrive/College /Human Pose Estimation Using Open CV/Images/Input/*.jpg')

In [3]:
inWidth = 452
inHeight = 452
thr = 0.2

BODY_PARTS = { "Nose": 0, "Neck": 1, "Right Shoulder": 2, "Right Elbow": 3, "Right Wrist": 4,
               "Left Shoulder": 5, "Left Elbow": 6, "Left Wrist": 7, "Right Hip": 8, "Right Knee": 9,
               "Right Ankle": 10, "Left Hip": 11, "Left Knee": 12, "Left Ankle": 13, "Right Eye": 14,
               "Left Eye": 15, "Right Ear": 16, "Left Ear": 17, "Background": 18 }

POSE_PAIRS = [ ["Neck", "Right Shoulder"], ["Neck", "Left Shoulder"], ["Right Shoulder", "Right Elbow"],
               ["Right Elbow", "Right Wrist"], ["Left Shoulder", "Left Elbow"], ["Left Elbow", "Left Wrist"],
               ["Neck", "Right Hip"], ["Right Hip", "Right Knee"], ["Right Knee", "Right Ankle"], ["Neck", "Left Hip"],
               ["Left Hip", "Left Knee"], ["Left Knee", "Left Ankle"], ["Neck", "Nose"], ["Nose", "Right Eye"],
               ["Right Eye", "Right Ear"], ["Nose", "Left Eye"], ["Left Eye", "Left Ear"] ]

def pose_estimation(frame):
    quad={}
    frameWidth=frame.shape[1]
    frameHeight=frame.shape[0]
    net.setInput(cv.dnn.blobFromImage(frame,1.0,(inWidth,inHeight),(127.5,127.5,127.5),swapRB=True, crop=False))
    out = net.forward()
    out = out[:, :19, :, :]  # output [1, 57, -1, -1], we only need the first 19 elements

    assert(len(BODY_PARTS) == out.shape[1])

    points = [] # Key Points
    for i in range(len(BODY_PARTS)):
        # Slice heatmap of corresponging body's part.
        heatMap = out[0, i, :, :]

        # Originally, we try to find all the local maximums. To simplify a sample
        # we just find a global one. However only a single pose at the same time
        # could be detected this way.
        _, conf, _, point = cv.minMaxLoc(heatMap)
        x = (frameWidth * point[0]) / out.shape[3]
        y = (frameHeight * point[1]) / out.shape[2]
        # Add a point if it's confidence is higher than threshold.
        points.append((int(x), int(y)) if conf > thr else None)

    for pair in POSE_PAIRS:
        partFrom = pair[0]
        partTo = pair[1]
        assert(partFrom in BODY_PARTS)
        assert(partTo in BODY_PARTS)

        idFrom = BODY_PARTS[partFrom]
        idTo = BODY_PARTS[partTo]

        if points[idFrom] and points[idTo]:
            quad.__setitem__(partFrom, points[idFrom])
            quad.__setitem__(partTo,points[idTo])
    return points,quad

if points[idFrom] and points[idTo]:
            cood.__setitem__(partFrom,pt[idFrom]);
            cood.__setitem__(partTo,pt[idTo]);

In [4]:
count=1
fields = ['Joint Name','Coordinates X','Coordinates Y']
for file in path:
    temp=[]
    print('Read From File :-' ,file)
    img = cv.imread(file)
    points,cord=pose_estimation(img)
    cord=OrderedDict(sorted(cord.items()))
    #print(points)
    #print(cord)
    take=file.replace('jpg','csv').replace('Input','Estimated Coordinates')
    #print(take);
    with open(take, 'w') as csvfile: 
        # creating a csv writer object 
        csvwriter = csv.writer(csvfile) 
         
        # writing the fields 
        csvwriter.writerow(fields) 
            
        # writing the data rows 
        for key, val in cord.items():
            cd=[]
            cd.extend(val)
            csvwriter.writerow([key, cd[0],cd[1]])
            del cd
    #print(points)
    #print(cord)
    count+=1 

Read From File :- /content/Google_Drive/MyDrive/College /Human Pose Estimation Using Open CV/Images/Input/1.jpg
Read From File :- /content/Google_Drive/MyDrive/College /Human Pose Estimation Using Open CV/Images/Input/2.jpg
Read From File :- /content/Google_Drive/MyDrive/College /Human Pose Estimation Using Open CV/Images/Input/3.jpg
Read From File :- /content/Google_Drive/MyDrive/College /Human Pose Estimation Using Open CV/Images/Input/5.jpg
Read From File :- /content/Google_Drive/MyDrive/College /Human Pose Estimation Using Open CV/Images/Input/4.jpg
